In [3]:
import cdms2 as cdms
import MV2 as MV
import genutil,cdutil,cdtime
from eofs.cdms import Eof
from eofs.multivariate.cdms import MultivariateEof
import glob
import string

import numpy as np
import matplotlib.pyplot as plt

import sys,os
sys.path.append("/Users/kmarvel/Google Drive/python-utils")

import CMIP5_tools as cmip5
import DA_tools
import Plotting

cmip_directory = "/Users/kmarvel/Google Drive/Sahel/DATA/CMIP5/"
daily_directory = "/Volumes/SahelData/Sahel_daily/"

### Set classic Netcdf (ver 3)
cdms.setNetcdfShuffleFlag(0)
cdms.setNetcdfDeflateFlag(0)
cdms.setNetcdfDeflateLevelFlag(0)

In [2]:
def insensitive_glob(pattern):
    def either(c):
        return '[%s%s]' % (c.lower(), c.upper()) if c.isalpha() else c
    return glob.glob(''.join(map(either, pattern)))

In [3]:
def get_piC(X,nyears=None):
    X.getAxis(0).designateTime()
    if X.getTime().units.find("days")<=0:
        newax = cdms.createAxis(X.getTime()[:]*30)
        newax.units = 'days since 1960-1-1'
        newax.designateTime()
        newax.setCalendar(cdtime.Calendar360)
        newax.id='time'
        X.setAxis(0,newax)
    cdutil.setTimeBoundsMonthly(X)
    if nyears is not None:
        return X[:nyears*12]
    else:
        return X
    
def get_rcp(X):
    X.getAxis(0).designateTime()
    if X.getTime().units.find("days")<=0:
        newax = cdms.createAxis(X.getTime()[:]*30)
        newax.units = 'days since 1960-1-1'
        newax.designateTime()
        newax.setCalendar(cdtime.Calendar360)
        newax.id='time'
        X.setAxis(0,newax)
    cdutil.setTimeBoundsMonthly(X)
    
    return X(time=('2006-1-1','2099-12-31'))

def get_historical(X):

    X.getAxis(0).designateTime()
    if X.getTime().units.find("days")<=0:
        newax = cdms.createAxis(X.getTime()[:]*30)
        newax.units = 'days since 1960-1-1'
        newax.designateTime()
        newax.setCalendar(cdtime.Calendar360)
        newax.id='time'
    X.setAxis(0,newax)
    
    cdutil.setTimeBoundsMonthly(X)
    
    return X(time=('1900-1-1','2005-12-31'))

# Historical + RCP85
Splice together historical and RCP8.5 runs

In [4]:
fmonthly=cdms.open(cmip_directory+"cmip5.sahel_precip.historical-rcp85.nc")
meast=fmonthly("pr_CE")
mwest = fmonthly("pr_W")
fmonthly.close()

splicedmodels=cmip5.models(meast)
okmodels=[]
okrips=[]
rcpok=[]
histok=[]
tmp_monthly_indices = []
counter = 0
for splicedmodel in splicedmodels:
    hmod,rmod=splicedmodel.split(" SPLICED WITH ")
    modelid=hmod.split(".")[1]
    rip=hmod.split(".")[3]
    #find the corresponding files in both historical and rcp
    histfileeast=insensitive_glob(daily_directory+"historical/"+modelid+"_*"+rip+"*East*")
    if histfileeast!=[]:
        histok+=[histfileeast[0]]
        
    #westhistok=len(insensitive_glob(daily_directory+"historical/"+modelid+"*"+rip+"*West*"))==1
    rcpfileeast=insensitive_glob(daily_directory+"rcp85/"+modelid+"*"+rip+"*East*")
    if rcpfileeast != []:
        rcpok+=[rcpfileeast[0]]
    if ((histfileeast != []) and  (rcpfileeast != [])):   
        okmodels+=[modelid+"*"+rip]
        tmp_monthly_indices +=[counter]
    counter += 1
    
monthly_indices = []
reallyok=[]
counter = 0
for x in okmodels:


    histfileeast=insensitive_glob(daily_directory+"historical/"+x+"*East*")[0]
    rcpfileeast=insensitive_glob(daily_directory+"rcp85/"+x+"*East*")[0]
    if ((histfileeast!=[]) and  (rcpfileeast != [])):
        fh=cdms.open(histfileeast)
        historicaldata=get_historical(fh("flag"))
        fh.close()
        fr=cdms.open(rcpfileeast)
        rcpdata=get_rcp(fr("flag"))
        splicetest=MV.concatenate((historicaldata,rcpdata),axis=0)
        fr.close()
        if len(splicetest)== 2400:
            reallyok+=[x]
            monthly_indices += [tmp_monthly_indices[counter]]
    counter += 1
EAST=MV.zeros((len(reallyok),200*12)) 
WEST=MV.zeros((len(reallyok),200*12)) 
i=0
for x in reallyok:
    histfileeast=insensitive_glob(daily_directory+"historical/"+x+"*East*")[0]
    rcpfileeast=insensitive_glob(daily_directory+"rcp85/"+x+"*East*")[0]
    if ((histfileeast!=[]) and  (rcpfileeast != [])):
        fh=cdms.open(histfileeast)
        historicaldata=get_historical(fh("flag"))
        fh.close()
        fr=cdms.open(rcpfileeast)
        rcpdata=get_rcp(fr("flag"))
        splicetest=MV.concatenate((historicaldata,rcpdata),axis=0)
        fr.close()
        EAST[i]=splicetest
        i+=1
i=0
for x in reallyok:
    histfilewest=insensitive_glob(daily_directory+"historical/"+x+"*West*")[0]
    rcpfilewest=insensitive_glob(daily_directory+"rcp85/"+x+"*West*")[0]
    if ((histfilewest!=[]) and  (rcpfilewest != [])):
        fh=cdms.open(histfilewest)
        historicaldata=get_historical(fh("flag"))
        fh.close()
        fr=cdms.open(rcpfilewest)
        rcpdata=get_rcp(fr("flag"))
        splicetest=MV.concatenate((historicaldata,rcpdata),axis=0)
        fr.close()
        WEST[i]=splicetest
        i+=1
        

monthly_indices=np.array(monthly_indices)
final_models=np.array(splicedmodels)[monthly_indices]
modax = cmip5.make_model_axis(final_models)
tax = splicetest.getTime()

EAST.setAxis(0,modax)
EAST.setAxis(1,tax)
EAST.id = "intensity_CE"
EAST.name="intensity_CE"
EAST.longname="Sahel precipitation intensity east of 0E"

WEST.setAxis(0,modax)
WEST.setAxis(1,tax)
WEST.id = "intensity_W"
WEST.name="intensity_W"
WEST.longname="Sahel precipitation intensity west of 0E"

cdutil.setTimeBoundsMonthly(EAST)  
cdutil.setTimeBoundsMonthly(WEST)

meast_trunc = MV.array(meast(time=('1900-1-1','2100-1-1')).asma()[monthly_indices])
meast_trunc.setAxis(0,modax)
meast_trunc.setAxis(1,tax)
meast_trunc.id="pr_CE"
meast_trunc.name="Sahel precipitation east of 0E"

mwest_trunc = MV.array(mwest(time=('1900-1-1','2100-1-1')).asma()[monthly_indices])
mwest_trunc.setAxis(0,modax)
mwest_trunc.setAxis(1,tax)
mwest_trunc.id = "pr_W"
mwest_trunc.name="pr_W"
mwest_trunc.longname="Sahel precipitation  west of 0E"

fw=cdms.open("/Users/kmarvel/Google Drive/Sahel/DATA/CMIP5/INTENSITY/cmip5.sahel_precip_intensity.historical-rcp85.nc","w")

fw.write(EAST)
fw.write(WEST)
fw.write(meast_trunc)
fw.write(mwest_trunc)
fw.close()

/Users/kmarvel/anaconda2/envs/cdat2/lib/python2.7/site-packages/numpy/ma/core.py:3174: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  dout = self.data[indx]
/Users/kmarvel/anaconda2/envs/cdat2/lib/python2.7/site-packages/numpy/ma/core.py:3206: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  mout = _mask[indx]


# PiControl
Collect all piControl runs with the relevant data and prepare for concatenation

In [5]:

fmonthlyc=cdms.open(cmip_directory+"cmip5.sahel_precip.piControl.nc")
cmeast=fmonthlyc("pr_CE")
cmwest = fmonthlyc("pr_W")

fmonthlyc.close()


piCmodels=cmip5.models(cmeast)
okmodels=[]
okrips=[]
rcpok=[]
histok=[]
tmp_monthly_indices = []
counter = 0
for piCmodel in piCmodels:
    
    modelid=piCmodel.split(".")[1]
    rip=piCmodel.split(".")[3]
    #find the corresponding file in piControl
    piCfileeast=insensitive_glob(daily_directory+"piControl/"+modelid+"_*"+rip+"*East*")
    
    if (piCfileeast != []):   
        okmodels+=[modelid+"*"+rip]
        tmp_monthly_indices +=[counter]
    counter += 1
    
monthly_indices = []
reallyok=[]
counter = 0
for x in okmodels:


    piCfileeast=insensitive_glob(daily_directory+"piControl/"+x+"*East*")[0]
    
    if (piCfileeast!=[]):
        fc=cdms.open(piCfileeast)
        piCdata=get_piC(fc("flag"),nyears=200)
        
        fc.close()
        
        if len(piCdata)== 2400:
            reallyok+=[x]
            monthly_indices += [tmp_monthly_indices[counter]]
    counter += 1
cEAST=MV.zeros((len(reallyok),200*12)) 
cWEST=MV.zeros((len(reallyok),200*12)) 
i=0
for x in reallyok:
    piCfileeast=insensitive_glob(daily_directory+"piControl/"+x+"*East*")[0]
    fc=cdms.open(piCfileeast)
    piCdataeast=get_piC(fc("flag"),nyears=200)
    tax=piCdataeast.getTime()
    
    fc.close()
    cEAST[i]=piCdataeast
        
    piCfilewest=insensitive_glob(daily_directory+"piControl/"+x+"*West*")[0]
    fc=cdms.open(piCfilewest)
    piCdatawest=get_piC(fc("flag"),nyears=200)
    fc.close()
    cWEST[i]=piCdatawest
    i+=1

cWEST.setAxis(1,tax)
cEAST.setAxis(1,tax)

monthly_indices=np.array(monthly_indices)
final_models=np.array(piCmodels)[monthly_indices]
modax = cmip5.make_model_axis(final_models)

cEAST.setAxis(0,modax)
cEAST.setAxis(1,tax)
cEAST.id = "intensity_CE"
cEAST.name="intensity_CE"
cEAST.longname="Sahel precipitation intensity east of 0E"

cWEST.setAxis(0,modax)
cWEST.setAxis(1,tax)
cWEST.id = "intensity_W"
cWEST.name="intensity_W"
cWEST.longname="Sahel precipitation intensity west of 0E"

cdutil.setTimeBoundsMonthly(cEAST)  
cdutil.setTimeBoundsMonthly(cWEST)

cmeast_trunc = MV.array(cmeast.asma()[monthly_indices])
cmeast_trunc.setAxis(0,modax)
cmeast_trunc.setAxis(1,tax)
cmeast_trunc.id="pr_CE"
cmeast_trunc.name="Sahel precipitation east of 0E"

cmwest_trunc = MV.array(cmwest.asma()[monthly_indices])
cmwest_trunc.setAxis(0,modax)
cmwest_trunc.setAxis(1,tax)
cmwest_trunc.id = "pr_W"
cmwest_trunc.name="pr_W"

fw=cdms.open("/Users/kmarvel/Google Drive/Sahel/DATA/CMIP5/INTENSITY/cmip5.sahel_precip_intensity.piControl.nc","w")

fw.write(cEAST)
fw.write(cWEST)
fw.write(cmeast_trunc)
fw.write(cmwest_trunc)
fw.close()

In [6]:
fmonthly=cdms.open(cmip_directory+"cmip5.sahel_precip.AA.nc")
meast=fmonthly("pr_CE")
mwest = fmonthly("pr_W")
fmonthly.close()

AAmodels=cmip5.models(meast)
okmodels=[]
okrips=[]

tmp_monthly_indices = []
counter = 0
for AAmodel in AAmodels:
    #hmod,rmod=AAmodel.split(" AA WITH ")
    modelid=AAmodel.split(".")[1]
    rip=AAmodel.split(".")[3]
    #find the corresponding files in both historical and rcp
    histfileeast=insensitive_glob(daily_directory+"historicalMisc/"+modelid+"_*"+rip+"*East*")
    if histfileeast!=[]:
  
        okmodels+=[modelid+"*"+rip]
        tmp_monthly_indices +=[counter]
    counter += 1
    
monthly_indices = []
reallyok=[]
counter = 0
for x in okmodels:


    histfileeast=insensitive_glob(daily_directory+"historicalMisc/"+x+"*R1**East*")[0]
    
    if (histfileeast!=[]):
        fh=cdms.open(histfileeast)
        historicaldata=get_historical(fh("flag"))
        fh.close()
        
        if len(historicaldata)== 106*12:
            reallyok+=[x]
            monthly_indices += [tmp_monthly_indices[counter]]
    counter += 1
EAST=MV.zeros((len(reallyok),106*12)) 
WEST=MV.zeros((len(reallyok),106*12)) 
i=0
for x in reallyok:
    histfileeast=insensitive_glob(daily_directory+"historicalMisc/"+x+"*R1*East*")[0]
    
    if (histfileeast!=[]):
        fh=cdms.open(histfileeast)
        historicaldata=get_historical(fh("flag"))
        fh.close()
        
        EAST[i]=historicaldata
        i+=1
i=0
for x in reallyok:
    histfilewest=insensitive_glob(daily_directory+"historicalMisc/"+x+"*R1*West*")[0]
    
    if (histfilewest!=[]) :
        fh=cdms.open(histfilewest)
        historicaldata=get_historical(fh("flag"))
        fh.close()
        
        WEST[i]=historicaldata
        i+=1
        

monthly_indices=np.array(monthly_indices)
final_models=np.array(AAmodels)[monthly_indices]
modax = cmip5.make_model_axis(final_models)
tax = historicaldata.getTime()

EAST.setAxis(0,modax)
EAST.setAxis(1,tax)
EAST.id = "intensity_CE"
EAST.name="intensity_CE"
EAST.longname="Sahel precipitation intensity east of 0E"

WEST.setAxis(0,modax)
WEST.setAxis(1,tax)
WEST.id = "intensity_W"
WEST.name="intensity_W"
WEST.longname="Sahel precipitation intensity west of 0E"

cdutil.setTimeBoundsMonthly(EAST)  
cdutil.setTimeBoundsMonthly(WEST)

meast_trunc = MV.array(meast(time=('1900-1-1','2100-1-1')).asma()[monthly_indices])
meast_trunc.setAxis(0,modax)
meast_trunc.setAxis(1,tax)
meast_trunc.id="pr_CE"
meast_trunc.name="Sahel precipitation east of 0E"

mwest_trunc = MV.array(mwest(time=('1900-1-1','2100-1-1')).asma()[monthly_indices])
mwest_trunc.setAxis(0,modax)
mwest_trunc.setAxis(1,tax)
mwest_trunc.id = "pr_W"
mwest_trunc.name="pr_W"
mwest_trunc.longname="Sahel precipitation  west of 0E"

fw=cdms.open("/Users/kmarvel/Google Drive/Sahel/DATA/CMIP5/INTENSITY/cmip5.sahel_precip_intensity.AA.nc","w")

fw.write(EAST)
fw.write(WEST)
fw.write(meast_trunc)
fw.write(mwest_trunc)
fw.close()

# Observations: CRU

In [38]:
fcru=cdms.open("/Users/kmarvel/Google Drive/Sahel/DATA/OBS/PROCESSED/CRU.nc")

In [42]:
feast=cdms.open("/Users/kmarvel/Google Drive/Sahel/DATA/OBS/DAILY/CRU_TS4p01_monthly_wet_1901-2016_EastSahel.nc")
pr_CE=fcru("pr_CE")
EAST=feast("wet")
cmip5.start_time(pr_CE)

meast_trunc=pr_CE[:len(EAST)]
EAST.setAxis(0,meast_trunc.getAxis(0))
feast.close()


meast_trunc.id="pr_CE"
meast_trunc.name="Sahel precipitation east of 0E"

EAST.id = "intensity_CE"
EAST.name="intensity_CE"
EAST.longname="Sahel precipitation intensity east of 0E"

In [43]:
fwest=cdms.open("/Users/kmarvel/Google Drive/Sahel/DATA/OBS/DAILY/CRU_TS4p01_monthly_wet_1901-2016_WestSahel.nc")
pr_W=fcru("pr_W")
WEST=fwest("wet")
cmip5.start_time(pr_W)

mwest_trunc=pr_CE[:len(WEST)]
WEST.setAxis(0,mwest_trunc.getAxis(0))
fwest.close()
mwest_trunc.id = "pr_W"
mwest_trunc.name="pr_W"
mwest_trunc.longname="Sahel precipitation  west of 0E"

WEST.id = "intensity_W"
WEST.name="intensity_W"
WEST.longname="Sahel precipitation intensity west of 0E"

In [44]:
fw=cdms.open("/Users/kmarvel/Google Drive/Sahel/DATA/CMIP5/INTENSITY/cmip5.sahel_precip_intensity.CRU.nc","w")

fw.write(EAST)
fw.write(WEST)
fw.write(meast_trunc)
fw.write(mwest_trunc)
fw.close()